In [10]:
from pycaret.classification import *
import numpy as np
import pandas as pd
import tensorflow as tf

#importing datasets
#I will be using the two smaller datasets provided for training and testing (explain why I used these in the thesis)
pathTesting = 'C:\\Users\\Jovana\\PycharmProjects\\neural_network_DNN\\include\\UNSW-NB15\\UNSW_NB15_testing-set.csv'
pathTraining= 'C:\\Users\\Jovana\\PycharmProjects\\neural_network_DNN\\include\\UNSW-NB15\\UNSW_NB15_training-set.csv'

ds_train=pd.read_csv(pathTraining)
ds_test=pd.read_csv(pathTesting)

ds=pd.concat([ds_train,ds_test], axis=0)

In [11]:
ds

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,1.415894e+04,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8.395112e+03,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1.572272e+03,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2.740179e+03,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8.561499e+03,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,udp,-,INT,2,0,104,0,200000.005100,254,0,8.320000e+07,0.000000,0,0,0.005000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,52,0,0,0,1,2,2,1,1,2,0,0,0,2,1,0,Normal,0
82328,82329,1.106101,tcp,-,FIN,20,8,18062,354,24.410067,254,252,1.241044e+05,2242.109863,7,1,55.880051,143.700000,4798.130981,190.980813,255,1072535109,3284291478,255,0.173208,0.100191,0.073017,903,44,0,0,1,1,2,1,1,1,0,0,0,3,2,0,Normal,0
82329,82330,0.000000,arp,-,INT,1,0,46,0,0.000000,0,0,0.000000e+00,0.000000,0,0,60000.720000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,0
82330,82331,0.000000,arp,-,INT,1,0,46,0,0.000000,0,0,0.000000e+00,0.000000,0,0,60000.732000,0.000000,10.954518,0.000000,0,0,0,0,0.000000,0.000000,0.000000,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,0


In [12]:
#there are no missing values, the feature 'service' has - where there is no service, rename it to 'undefined'
ds['service'].replace('-','undefined',inplace=True)

In [13]:
ds['label'].replace(1,'attack',inplace=True)
ds['label'].replace(0, 'normal', inplace=True)

In [14]:
ds

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,undefined,FIN,6,4,258,172,74.087490,252,254,1.415894e+04,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,normal
1,2,0.649902,tcp,undefined,FIN,14,38,734,42014,78.473372,62,252,8.395112e+03,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,normal
2,3,1.623129,tcp,undefined,FIN,8,16,364,13186,14.170161,62,252,1.572272e+03,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,normal
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2.740179e+03,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,normal
4,5,0.449454,tcp,undefined,FIN,10,6,534,268,33.373826,254,252,8.561499e+03,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,udp,undefined,INT,2,0,104,0,200000.005100,254,0,8.320000e+07,0.000000,0,0,0.005000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,52,0,0,0,1,2,2,1,1,2,0,0,0,2,1,0,Normal,normal
82328,82329,1.106101,tcp,undefined,FIN,20,8,18062,354,24.410067,254,252,1.241044e+05,2242.109863,7,1,55.880051,143.700000,4798.130981,190.980813,255,1072535109,3284291478,255,0.173208,0.100191,0.073017,903,44,0,0,1,1,2,1,1,1,0,0,0,3,2,0,Normal,normal
82329,82330,0.000000,arp,undefined,INT,1,0,46,0,0.000000,0,0,0.000000e+00,0.000000,0,0,60000.720000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,normal
82330,82331,0.000000,arp,undefined,INT,1,0,46,0,0.000000,0,0,0.000000e+00,0.000000,0,0,60000.732000,0.000000,10.954518,0.000000,0,0,0,0,0.000000,0.000000,0.000000,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,normal


In [60]:
pd.unique(ds.proto.values)

array(['tcp', 'udp', 'arp', 'ospf', 'icmp', 'igmp', 'rtp', 'ddp',
       'ipv6-frag', 'cftp', 'wsn', 'pvp', 'wb-expak', 'mtp', 'pri-enc',
       'sat-mon', 'cphb', 'sun-nd', 'iso-ip', 'xtp', 'il', 'unas',
       'mfe-nsp', '3pc', 'ipv6-route', 'idrp', 'bna', 'swipe',
       'kryptolan', 'cpnx', 'rsvp', 'wb-mon', 'vmtp', 'ib', 'dgp',
       'eigrp', 'ax.25', 'gmtp', 'pnni', 'sep', 'pgm', 'idpr-cmtp',
       'zero', 'rvd', 'mobile', 'narp', 'fc', 'pipe', 'ipcomp', 'ipv6-no',
       'sat-expak', 'ipv6-opts', 'snp', 'ipcv', 'br-sat-mon', 'ttp',
       'tcf', 'nsfnet-igp', 'sprite-rpc', 'aes-sp3-d', 'sccopmce', 'sctp',
       'qnx', 'scps', 'etherip', 'aris', 'pim', 'compaq-peer', 'vrrp',
       'iatp', 'stp', 'l2tp', 'srp', 'sm', 'isis', 'smp', 'fire', 'ptp',
       'crtp', 'sps', 'merit-inp', 'idpr', 'skip', 'any', 'larp', 'ipip',
       'micp', 'encap', 'ifmp', 'tp++', 'a/n', 'ipv6', 'i-nlsp',
       'ipx-n-ip', 'sdrp', 'tlsp', 'gre', 'mhrp', 'ddx', 'ippc', 'visa',
       'secure-vmtp', 

In [15]:
#drop the id column
ds=ds.drop('id', axis=1)

In [16]:
#first binary classification, for that we will drop the attack cat
ds_binary=ds.drop('attack_cat',axis=1)

In [123]:
dsp_tr_binary=setup(data = ds_binary,
             target = 'label',train_size=0.8,
             numeric_imputation = 'mean',
             categorical_features = ['proto','service','state'],
             numeric_features=['dur','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm',
                               'ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports'],
             normalize=True,normalize_method='minmax',
             feature_selection=True,feature_selection_threshold=0.2,
             silent = True, fold_shuffle=True, remove_multicollinearity=True)

,Description,Value
0,session_id,3704
1,Target,label
2,Target Type,Binary
3,Label Encoded,"attack: 0, normal: 1"
4,Original Data,"(257673, 43)"
5,Missing Values,False
6,Numeric Features,39
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [124]:
X_train_bin=get_config('X_train')
X_test_bin=get_config('X_test')
Y_train_bin=get_config('y_train')
Y_test_bin=get_config('y_test')

In [125]:
X_train_bin

,dload,dinpkt,service_ssh,stcpb,proto_sun-nd,djit,state_REQ,service_dns,proto_sctp,rate,...,swin,response_body_len,proto_udp,service_undefined,dtcpb,proto_unas,service_http,proto_mobile,sload,service_ftp-data
139738,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.200000,...,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,1.628572e-02,0.0
43869,0.001166,6.416928e-04,0.0,0.385816,0.0,0.000116,0.0,0.0,0.0,0.000063,...,1.0,0.000012,0.0,0.0,0.635662,0.0,1.0,0.0,4.212143e-06,0.0
71803,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.333333,...,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0,4.761905e-02,0.0
116658,0.000415,7.399206e-04,0.0,0.086627,0.0,0.000106,0.0,0.0,0.0,0.000047,...,1.0,0.000000,0.0,1.0,0.179906,0.0,0.0,0.0,1.933541e-06,0.0
104651,0.000886,9.424858e-04,0.0,0.867988,0.0,0.000137,0.0,0.0,0.0,0.000036,...,1.0,0.000000,0.0,1.0,0.329263,0.0,0.0,0.0,1.393886e-06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129967,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.250000,...,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,2.035714e-02,0.0
75078,0.004361,1.108914e-03,0.0,0.400822,0.0,0.010738,0.0,0.0,0.0,0.000023,...,1.0,0.000000,0.0,1.0,0.334589,0.0,0.0,0.0,5.992443e-07,0.0
109458,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.333333,...,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0,4.761905e-02,0.0
126395,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.250000,...,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,2.035714e-02,0.0


In [140]:
#now we make and test the neural networks
from tensorflow.python.keras.layers import Activation
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.advanced_activations import PReLU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
act2 = PReLU(alpha_initializer='zeros', weights=None)
model2 = Sequential()
# define the NN architecture and activation function

model2.add(Dense(496, input_dim=40, activation=Activation(act2)))
model2.add(Dropout(0.1))
model2.add(Dense(248, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(124, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(62, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))

sgd =SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', optimizer=sgd, metrics= ['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001,patience=5) ## early stopping
model2.fit(X_train_bin, Y_train_bin, validation_data=(X_test_bin, Y_test_bin), epochs=100, batch_size=64, callbacks=[es])

Epoch 1/100
3221/3221 [==============================] - 12s 4ms/step - loss: 0.2014 - accuracy: 0.8933 - val_loss: 0.1765 - val_accuracy: 0.8996
Epoch 2/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1740 - accuracy: 0.9070 - val_loss: 0.1696 - val_accuracy: 0.9089
Epoch 3/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1685 - accuracy: 0.9113 - val_loss: 0.1647 - val_accuracy: 0.9107
Epoch 4/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1657 - accuracy: 0.9128 - val_loss: 0.1592 - val_accuracy: 0.9141
Epoch 5/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1630 - accuracy: 0.9149 - val_loss: 0.1605 - val_accuracy: 0.9187
Epoch 6/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1617 - accuracy: 0.9158 - val_loss: 0.1571 - val_accuracy: 0.9194
Epoch 7/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1601 - accuracy: 0.9173 - val_loss: 0.1537 -

In [141]:
from sklearn.metrics import accuracy_score
# or find the prediction accuracy using predict_classes (same as test accuracy)
y_pred2 = model2.predict_classes(X_test_bin)
pred_accuracy2 = accuracy_score(Y_test_bin, y_pred2)
pred_accuracy2

0.91974386339381

In [142]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
# Precision
precision = precision_score(Y_test_bin, y_pred2)
print('Precision : %f'% precision)

Precision : 0.909040


In [143]:
# Recall
recall = recall_score(Y_test_bin, y_pred2)
print('Recall: %f' % recall)

Recall: 0.866087


In [144]:
# f1-score
f1 = f1_score(Y_test_bin, y_pred2)
print('F1 score: %f' % f1)

F1 score: 0.887044


In [145]:
roc = roc_auc_score(Y_test_bin, y_pred2)
print('ROC_AUC Score: %f' % roc)

ROC_AUC Score: 0.908260


In [146]:
from sklearn.metrics import confusion_matrix

# confusion matrix
confusionmatrix = confusion_matrix(Y_test_bin, y_pred2)
print(confusionmatrix)

[[31159  1625]
 [ 2511 16240]]


In [134]:
ds_multi=pd.concat([ds_train,ds_test], axis=0)

In [135]:
ds_multi=ds_multi.drop('label',axis=1)

In [136]:
ds_multi=ds_multi.drop('id',axis=1)

In [137]:
ds_multi

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,...,1,1,1,0,0,0,1,1,0,Normal
1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,...,1,1,2,0,0,0,1,6,0,Normal
2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,...,1,1,3,0,0,0,2,6,0,Normal
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,...,1,1,3,1,1,0,2,1,0,Normal
4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,...,2,1,40,0,0,0,2,39,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,0.000005,udp,-,INT,2,0,104,0,200000.005100,254,...,1,1,2,0,0,0,2,1,0,Normal
82328,1.106101,tcp,-,FIN,20,8,18062,354,24.410067,254,...,1,1,1,0,0,0,3,2,0,Normal
82329,0.000000,arp,-,INT,1,0,46,0,0.000000,0,...,1,1,1,0,0,0,1,1,1,Normal
82330,0.000000,arp,-,INT,1,0,46,0,0.000000,0,...,1,1,1,0,0,0,1,1,1,Normal


In [138]:
pd.unique(ds_multi.attack_cat.values)

array(['Normal', 'Backdoor', 'Analysis', 'Fuzzers', 'Shellcode',
       'Reconnaissance', 'Exploits', 'DoS', 'Worms', 'Generic'],
      dtype=object)

In [161]:
dsp_tr_multi=setup(data = ds_multi,
             target = 'attack_cat',train_size=0.8,
             numeric_imputation = 'mean',
             categorical_features = ['proto','service','state'],
             numeric_features=['dur','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm',
                               'ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports'],
             normalize=True,normalize_method='minmax',
             feature_selection=True,feature_selection_threshold=0.2,
             silent = True, fold_shuffle=True, remove_multicollinearity=True)

,Description,Value
0,session_id,6310
1,Target,attack_cat
2,Target Type,Multiclass
3,Label Encoded,"Analysis: 0, Backdoor: 1, DoS: 2, Exploits: 3, Fuzzers: 4, Generic: 5, Normal: 6, Reconnaissance: 7, Shellcode: 8, Worms: 9"
4,Original Data,"(257673, 43)"
5,Missing Values,False
6,Numeric Features,39
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [162]:
X_train_multi=get_config('X_train')
X_test_multi=get_config('X_test')
Y_train_multi=get_config('y_train')
Y_test_multi=get_config('y_test')

In [148]:
X_train_multi

,service_-,dload,dinpkt,sloss,stcpb,djit,service_dns,state_FIN,proto_sctp,ackdat,...,state_INT,proto_ospf,response_body_len,proto_udp,dtcpb,proto_unas,service_http,service_ftp,sload,ct_flw_http_mthd
96429,1.0,0.000003,0.091624,0.001128,0.155309,0.035638,0.0,1.0,0.0,0.012130,...,0.0,0.0,0.0,0.0,0.601472,0.0,0.0,0.0,2.326462e-08,0.000000
61938,0.0,0.000198,0.001329,0.000376,0.239266,0.000314,0.0,1.0,0.0,0.008370,...,0.0,0.0,0.0,0.0,0.001439,0.0,1.0,0.0,3.115452e-06,0.033333
19265,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000,...,1.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,1.523046e-02,0.000000
99374,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.484450e-02,0.000000
99408,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,2.226675e-02,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166627,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000,...,1.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,1.087890e-02,0.000000
33536,1.0,0.017018,0.000019,0.000000,0.000000,0.000003,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,1.190666e-04,0.000000
54789,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,1.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,7.695390e-02,0.000000
14574,1.0,0.236462,0.000004,0.001128,0.291019,0.000029,0.0,1.0,0.0,0.000048,...,0.0,0.0,0.0,0.0,0.791589,0.0,0.0,0.0,3.927033e-04,0.000000


In [163]:
act3 = PReLU(alpha_initializer='zeros', weights=None)
model3 = Sequential()
# define the NN architecture and activation function

model3.add(Dense(496, input_dim=34, activation=Activation(act2)))
model3.add(Dropout(0.1))
model3.add(Dense(248, activation='relu'))
model3.add(Dropout(0.1))
model3.add(Dense(124, activation='relu'))
model3.add(Dropout(0.1))
model3.add(Dense(62, activation='relu'))
model3.add(Dropout(0.1))
model3.add(Dense(10))
model3.add(Activation('softmax'))

sgd =SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model3.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics= ['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001,patience=5) ## early stopping
model3.fit(X_train_multi, Y_train_multi, validation_data=(X_test_multi, Y_test_multi), epochs=100, batch_size=64, callbacks=[es])

Epoch 1/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.7006 - accuracy: 0.7248 - val_loss: 0.6095 - val_accuracy: 0.7593
Epoch 2/100
3221/3221 [==============================] - 10s 3ms/step - loss: 0.6244 - accuracy: 0.7460 - val_loss: 0.5952 - val_accuracy: 0.7617
Epoch 3/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.6088 - accuracy: 0.7520 - val_loss: 0.5945 - val_accuracy: 0.7594
Epoch 4/100
3221/3221 [==============================] - 10s 3ms/step - loss: 0.5990 - accuracy: 0.7553 - val_loss: 0.5714 - val_accuracy: 0.7719
Epoch 5/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.5931 - accuracy: 0.7589 - val_loss: 0.5703 - val_accuracy: 0.7674
Epoch 6/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.5858 - accuracy: 0.7606 - val_loss: 0.5741 - val_accuracy: 0.7716
Epoch 7/100
3221/3221 [==============================] - 10s 3ms/step - loss: 0.5808 - accuracy: 0.7650 - val_loss: 0.5569 -

In [164]:
# confusion matrix
y_pred3 = model3.predict_classes(X_test_multi)
confusionmatrix3 = confusion_matrix(Y_test_multi, y_pred3)
print(confusionmatrix3)

[[    8     0     0   421     1     0   102     0     0     0]
 [    0     6     6   412     0     0    19    24     0     0]
 [    0     5    92  2958    23     9   110    32    13     0]
 [    0     1    26  8383   118     6   343    58    15     0]
 [    0     0     1   728  1686     0  2175   113    40     0]
 [    0     0    19   211    31 11505    19     8     5     0]
 [    0     0     3   239   686     2 17547   170    31     0]
 [    0     0     9   827     6     0   334  1615     0     0]
 [    0     0     0    45    23     1    64    90    76     0]
 [    0     0     0    27     3     0     4     0     1     0]]


In [165]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix
target_names=['Analysis','Backdoor','DoS','Exploits','Fuzzers','Generic','Normal','Recconnaissanse','Shellcode','Worms']
print(classification_report(Y_test_multi, y_pred3, target_names=target_names))

                 precision    recall  f1-score   support

       Analysis       1.00      0.02      0.03       532
       Backdoor       0.50      0.01      0.03       467
            DoS       0.59      0.03      0.05      3242
       Exploits       0.59      0.94      0.72      8950
        Fuzzers       0.65      0.36      0.46      4743
        Generic       1.00      0.98      0.99     11798
         Normal       0.85      0.94      0.89     18678
Recconnaissanse       0.77      0.58      0.66      2791
      Shellcode       0.42      0.25      0.32       299
          Worms       0.00      0.00      0.00        35

       accuracy                           0.79     51535
      macro avg       0.64      0.41      0.41     51535
   weighted avg       0.79      0.79      0.76     51535



In [166]:
confusionmatrix4 = multilabel_confusion_matrix(Y_test_multi, y_pred3)
print(confusionmatrix4)

[[[51003     0]
  [  524     8]]

 [[51062     6]
  [  461     6]]

 [[48229    64]
  [ 3150    92]]

 [[36717  5868]
  [  567  8383]]

 [[45901   891]
  [ 3057  1686]]

 [[39719    18]
  [  293 11505]]

 [[29687  3170]
  [ 1131 17547]]

 [[48249   495]
  [ 1176  1615]]

 [[51131   105]
  [  223    76]]

 [[51500     0]
  [   35     0]]]
